In [1]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions

In [2]:
spark = SparkSession \
    .builder \
    .appName("Market Analytics") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

## Load dataset

In [3]:
df = spark.read.format("csv").option("header", "true").load("dataset/tokopedia.csv")
df.show()

+--------------------+---------+---------+---------+--------------------+--------------------+
|          breadcrumb|item_sold|      pid|    price|               title|                 url|
+--------------------+---------+---------+---------+--------------------+--------------------+
|Dapur ;Bekal ;Cup...|        2|389668528|   70.000|Mayonnaise & Ketc...|https://www.tokop...|
|Rumah Tangga ;Keb...|        0|390053120|  160.000|BOLDe Super BROOM...|https://www.tokop...|
|Perawatan Hewan ;...|        0|390055891|1.100.000|Two Tone Brown (S...|https://www.tokop...|
|Fashion Muslim ;P...|        0|389610348|  325.000|MUKENA ALDIVA har...|https://www.tokop...|
|     Produk Lainnya |        1|389666117|  179.900|terpal kolam kota...|https://www.tokop...|
|Fashion Pria ;Aks...|        0|389609406|   58.000|TOPI TRUCKER CONS...|https://www.tokop...|
|Makanan & Minuman...|        0|389613659|  195.000|ARTE CHOCOLATE ( ...|https://www.tokop...|
|Perawatan Hewan ;...|        2|389671338|   10.00

In [4]:
df.dtypes

[('breadcrumb', 'string'),
 ('item_sold', 'string'),
 ('pid', 'string'),
 ('price', 'string'),
 ('title', 'string'),
 ('url', 'string')]

In [6]:
# some data have null in item_sold, pid, price, url
df = df.where(df['item_sold'].isNotNull())\
    .where(df['pid'].isNotNull())\
    .where(df['price'].isNotNull())\
    .where(df['title'].isNotNull())\
    .where(df['url'].isNotNull())

# remove wrong value
df = df.rdd.filter(lambda x: "http" not in x['item_sold'])\
    .filter(lambda x: "http" in x['url']).toDF()

In [ ]:
# extract more data


## Total Transaction (QTY)

In [112]:
from operator import add
df_clean.select('item_sold').rdd.map(lambda x: int(x['item_sold'])).reduce(add)

12298304

## Total Transaction (Rupiah)

In [113]:
def price(x):
    return int(x.replace(".", ""))

df_clean.select('price').rdd.map(lambda x: price(x['price'])).reduce(add)

862586607816

## Total Seller

In [114]:
def seller(x):
    return x.split("/")[3]

df_clean.select('url').rdd.map(lambda x: seller(x['url'])).distinct().count()

90144

## Total Product

In [115]:
df_clean.rdd.count()

1281288

## 10 Best Selling

In [118]:
# name, url, market_name, market_url, 
df_clean.rdd.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 75.0 failed 1 times, most recent failure: Lost task 0.0 in stage 75.0 (TID 179, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1853)
	at java.io.ObjectOutputStream.write(ObjectOutputStream.java:709)
	at org.apache.spark.util.Utils$.writeByteBuffer(Utils.scala:260)
	at org.apache.spark.scheduler.DirectTaskResult$$anonfun$writeExternal$1.apply$m

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 51518)
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/anaconda3/lib/python3.7/socketserver.py", line 344, in process_request
    self.finish_request(request, client_address)
  File "/anaconda3/lib/python3.7/socketserver.py", line 357, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/anaconda3/lib/python3.7/socketserver.py", line 717, in __init__
    self.handle()
  File "/anaconda3/lib/python3.7/site-packages/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/anaconda3/lib/python3.7/site-packages/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/anaconda3/lib/python3.7/site-packages/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(sel